# In this model, I will train the NBEATS NN model on all states in the USA, the USA, and other countries as well to forecast deaths and cases



## Furthermore, I will give the output metrics of two forecasts of two different regions around the world to display a couple examples.

In [188]:
import numpy as np
from nbeats_keras.model import NBeatsNet
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt

In [337]:
df = pd.read_csv('coviddata.csv')
df['Date'] = pd.to_datetime(df.Date)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Let's try to train the model on all areas inside the USA

In [329]:
#get list of all represented countries
countries = df['Country_Region'].unique()
countries = np.sort(countries)
provinces = df['Province_State'].unique()

**Since time series for NBEATS is univariate, we cannot make an additional feature which encodes the country and case type per country as another feature because then the time series is not univariate**

**So, we will create a function where the user inputs a specific country, forecast horizon, and the case type(confirmed or deaths) and the model will train accordingly**

**The function will save and return the trained model, which then can be used to perform predictions**

**The function will also return a flattened version of the data history, which we can plot with the predictions**

In [324]:
def train_for_country(countries, df, country, forecast_horizon, case, province=None):
    if province == None:
        #print(df.shape)
        current_df = df.query(f'Country_Region == "{country}" and Case_Type == "{case}"')
        #print(current_df.shape)
        sorteddf = current_df.sort_values(by='Date')
        sorteddf = sorteddf.groupby(sorteddf['Date'].dt.date).sum()
        #print(sorteddf.shape)
    else:
        current_df = df.query(f'Case_Type == "{case}" and Province_State=="{province}"')
        sorteddf = current_df.sort_values(by='Date')
        sorteddf = sorteddf.groupby(sorteddf['Date'].dt.date).sum()

    data=[]
    for j in range(len(sorteddf['Cases'])):
        if j+14 > len(sorteddf['Cases']):
            data.append([[sorteddf['Cases'][k]] for k in range (j,len(sorteddf['Cases']))])
        else:
            data.append([[sorteddf['Cases'][k]] for k in range (j,j+14)])

    while len(data[-1]) < 14:
        data.remove(data[-1])

    predictions=[]
    for j in range(13+forecast_horizon,len(sorteddf['Cases'])):
        predictions.append([[sorteddf['Cases'][j]]])

    data=np.array(data)
    predictions=np.array(predictions)
    data_plot = data.flatten()
    #x_arr = np.arange(len(data_plot))
    #plt.plot(x_arr, data_plot, label=f'{case} in {country} up till {forecast_horizon} days ago')


    future = np.array([data[j] for j in range(len(data)-forecast_horizon, len(data))]).astype('float32')
    #flat_future = future.flatten()
    #x_arr2 = np.arange(len(data_plot), len(data_plot) + len(flat_future))
    #plt.plot(x_arr2, flat_future, label = f'{case} in {country} for the last {forecast_horizon} days')

    for j in range(0,forecast_horizon):
        data = np.delete(data, -1,0)

    splitter = int(0.9*len(data))
    x_train, y_train, x_test, y_test = data[:splitter].astype('float32'), predictions[:splitter].astype('float32'), data[splitter:].astype('float32'), predictions[splitter:].astype('float32')

    #train
    num_samples, time_steps, input_dim, output_dim = len(data), 14, 1, 1

    model = NBeatsNet(backcast_length=time_steps, forecast_length=output_dim,
                  stack_types=(NBeatsNet.GENERIC_BLOCK, NBeatsNet.GENERIC_BLOCK,), nb_blocks_per_stack=12,
                  thetas_dim=(4, 4), share_weights_in_stack=True, hidden_layer_units=128)

    model.compile_model(loss='mape', learning_rate=1e-5)

    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=5)
    #plt.plot(model.history['loss'], label = 'mape loss')
    #plt.legend()
    #plt.show()

    model.save(f'n_beats_model.{country}.{case}')

    modelreturn = NBeatsNet.load(f'n_beats_model.{country}.{case}')

    #last_index_for_plotting_prediction = len(data_plot)+len(flat_future)

    #flat_pred = pred.flatten()
    #x_arr3 = np.arange(last_index_for_plotting_prediction, last_index_for_plotting_prediction+len(flat_pred))
    #plt.plot(x_arr3, flat_pred, label=f'prediction for next {forecast_horizon} days')
    #plt.legend()
    #plt.show()
        
    
    return modelreturn, future, data.flatten(), future.flatten()

**The prediction that is given here, with the command print(pred) are the predictions of COVID-19 cases in China starting tomorrow.**

In [320]:
my_model,future, history, for_tom = train_for_country(countries, df, "China", 7, "Confirmed")
pred = my_model.predict(future)

Model: "model_50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 14, 1)        0                                            
__________________________________________________________________________________________________
lambda_50 (Lambda)              (None, 14)           0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 128)          1920        lambda_50[0][0]                  
                                                                 subtract_1057[0][0]              
                                                                 subtract_1058[0][0]              
                                                                 subtract_1059[0][0]       

Train on 94 samples, validate on 11 samples
Epoch 1/20
94/94 [==============================] - 3s 30ms/step - loss: 432.8516 - val_loss: 355.9959
Epoch 2/20
94/94 [==============================] - 0s 2ms/step - loss: 232.3496 - val_loss: 153.3154
Epoch 3/20
94/94 [==============================] - 0s 2ms/step - loss: 65.1058 - val_loss: 27.4802
Epoch 4/20
94/94 [==============================] - 0s 2ms/step - loss: 21.3615 - val_loss: 11.9948
Epoch 5/20
94/94 [==============================] - 0s 2ms/step - loss: 10.0296 - val_loss: 2.1798
Epoch 6/20
94/94 [==============================] - 0s 2ms/step - loss: 7.3049 - val_loss: 1.0293
Epoch 7/20
94/94 [==============================] - 0s 2ms/step - loss: 6.7785 - val_loss: 0.0351
Epoch 8/20
94/94 [==============================] - 0s 2ms/step - loss: 6.1187 - val_loss: 3.3988
Epoch 9/20
94/94 [==============================] - 0s 2ms/step - loss: 6.3428 - val_loss: 3.1490
Epoch 10/20
94/94 [==============================] - 0s 2ms/

**Here are the predictions for cases starting tomorrow**

In [321]:
print(pred)

[[[84304.67 ]]

 [[84298.44 ]]

 [[84310.52 ]]

 [[84321.48 ]]

 [[84316.95 ]]

 [[84329.95 ]]

 [[84335.414]]]


**Currently, there are almost 83,000 cases of COVID-19 in China, so the predictions seem reliable for how the cases should grow starting tomorrow**

# Now, we will use the function on a state in the USA to show how predictions can be made per state

In [335]:
my_model,future, history, for_tom = train_for_country(countries, df, "US", 7, "Deaths", province="Connecticut")
pred = my_model.predict(future)

Model: "model_56"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_variable (InputLayer)     (None, 14, 1)        0                                            
__________________________________________________________________________________________________
lambda_56 (Lambda)              (None, 14)           0           input_variable[0][0]             
__________________________________________________________________________________________________
0/0/generic/d1 (Dense)          (None, 128)          1920        lambda_56[0][0]                  
                                                                 subtract_1201[0][0]              
                                                                 subtract_1202[0][0]              
                                                                 subtract_1203[0][0]       

Train on 94 samples, validate on 11 samples
Epoch 1/20
94/94 [==============================] - 3s 31ms/step - loss: 598558.2512 - val_loss: 14.3921
Epoch 2/20
94/94 [==============================] - 0s 2ms/step - loss: 183879.7266 - val_loss: 30.7116
Epoch 3/20
94/94 [==============================] - 0s 2ms/step - loss: 219130.8638 - val_loss: 31.7429
Epoch 4/20
94/94 [==============================] - 0s 2ms/step - loss: 583849.3152 - val_loss: 30.6459
Epoch 5/20
94/94 [==============================] - 0s 2ms/step - loss: 602291.2060 - val_loss: 23.4122
Epoch 6/20
94/94 [==============================] - 0s 2ms/step - loss: 116731.0344 - val_loss: 15.3726
Epoch 7/20
94/94 [==============================] - 0s 2ms/step - loss: 196256.1027 - val_loss: 16.0979
Epoch 8/20
94/94 [==============================] - 0s 2ms/step - loss: 303191.1476 - val_loss: 8.0219
Epoch 9/20
94/94 [==============================] - 0s 2ms/step - loss: 121939.5676 - val_loss: 4.1287
Epoch 10/20
94/94 [==

**These predictions are for deaths in Connecticut starting tomorrow. The current count is 3,769, and so the forecasts for deaths starting tomorrow seem reliable.**

In [336]:
print(pred)

[[[3925.454 ]]

 [[3935.7559]]

 [[3923.389 ]]

 [[3963.5498]]

 [[3960.1963]]

 [[4006.3765]]

 [[4166.7354]]]


# Focus of Improvements

**So far, this function predicts pretty well for forecasts around the world for confirmed cases and deaths. However, I am trying to definitely improve it in some areas.**

**The model right now gets very good forecasts as seen by the displayed examples, but it can also be inconsistent. Sometimes the forecasts are very good and sometimes they are not as good, especially depending on the region picked. I think this has something to do with the hyperparameters as well, and they maybe need to be fine tuned via some algorithm in the function according to certain data metric.**

**Furthermore, this function can be called on any country or one of the given provinces. All 50 states in the USA are covered in provinces, and some more around the world. Please look at the contents of "provinces" array at the top to see which ones are covered. Right now, this function can be used as a building block to perform more operations such as finding the average errors across states or countries, etc.**

**However, I plan to add in this functionality to make it more easy to use**